# Region Selection and Map Preview with Ipyleaflet

In [1]:
# Import the necessary libraries to format, send, and parse our returned results

import birdy
import json
import geopandas as gpd
import requests
import matplotlib
import ipyleaflet
import ipywidgets

In [2]:
# Ensure that the ipyleaflet extension is enbaled

!jupyter nbextension enable --py --sys-prefix ipyleaflet

Enabling notebook extension jupyter-leaflet/extension...
      - Validating: OK


In [3]:
# Create WPS instances

# # locally running instance
# raven_url = "http://localhost:9099/wps"
# raven = birdy.WPSClient(raven_url, progress=True)

# instance via pavics.ouranos.ca
pavics_url = "https://pavics.ouranos.ca"
raven_url = f'{pavics_url}/twitcher/ows/proxy/raven/wps'

raven = birdy.WPSClient(raven_url, progress=True)

In [4]:
# Build an interactive map with ipyleaflet

initial_lat_lon = (48.63, -74.71)

leaflet_map = ipyleaflet.Map(
    center=initial_lat_lon,
    basemap=ipyleaflet.basemaps.OpenTopoMap,
)

# Add a custom zoom slider
zoom_slider = ipywidgets.IntSlider(description='Zoom level:', min=1, max=10, value=5)
ipywidgets.jslink((zoom_slider, 'value'), (leaflet_map, 'zoom'))
widget_control1 = ipyleaflet.WidgetControl(widget=zoom_slider, position='topright')
leaflet_map.add_control(widget_control1)

# Add a marker to the map
marker = ipyleaflet.Marker(location=initial_lat_lon, draggable=True)
leaflet_map.add_layer(marker)

In [5]:
# Add an overlay widget

html = ipywidgets.HTML("""Hover over a feature!""")
html.layout.margin = '0px 10px 10px 10px'

control = ipyleaflet.WidgetControl(widget=html, position='bottomleft')
leaflet_map.add_control(control)

def update_html(feature,  **kwargs):
    html.value = '''
        <h2><b>USGS HydroBASINS</b></h2>
        <h4>ID: {}</h4>
        <h4>Upstream Area: {} sq. km.</h4> 
        <h4>Sub-basin Area: {} sq. km.</h4>
    '''.format(feature['properties']['id'],
               feature['properties']['UP_AREA'],
               feature['properties']['SUB_AREA'])

In [6]:
# Load the map in the notebook
leaflet_map

Map(basemap={'url': 'https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png', 'max_zoom': 17, 'attribution': 'Map da…

**Before continuing!**

Try dragging and placing the marker at the mouth of a river, over a large lake such as Lac Saint Jean (next to Alma, east of the initial marker position), or anywhere else within North America.

In [11]:
user_lonlat = list(reversed(marker.location))
user_lonlat

[-71.93847656250001, 48.48748647988415]

In [12]:
# user_location = raven.hydrobasins_select(location=lonlat, aggregate_upstream=True)
user_location = raven.hydrosheds_select(location=str(user_lonlat), aggregate_upstream=True)


In [13]:
# Before continuing, wait for the process to finish!

# Extract the URL of the resulting GeoJSON feature
user_shape = user_location.get(asobj=False).feature
user_shape

# Examine its properties
user_data = requests.get(user_shape).json()
user_data['properties']

{'COAST': 0,
 'DIST_MAIN': 167.2,
 'DIST_SINK': 167.2,
 'ENDO': 0,
 'HYBAS_ID': 7129001701,
 'LAKE': 301,
 'NEXT_DOWN': 7120317402,
 'NEXT_SINK': 7120034330,
 'ORDER': 1,
 'PFAF_ID': 724083034000,
 'SIDE': 'L',
 'SORT': 96070,
 'SUB_AREA': 72417.5,
 'UP_AREA': 1067.6,
 'id': 'USGS_HydroBASINS_lake_na_lev12.96070'}

In [14]:
# Add this GeoJSON to the map!
df = gpd.read_file(user_shape)
user_geojson = ipyleaflet.GeoData(geo_dataframe=df, 
    style = {
        'color': 'blue', 
        'opacity':1, 
        'weight':1.9, 
        'fillOpacity':0.5,
    },
                                  
    hover_style={'fillColor': '#b08a3e' , 'fillOpacity': 0.9}
)

leaflet_map.add_layer(user_geojson)

user_geojson.on_hover(update_html)